# Distributed Redshift 

### Introduction

As we know, redshift is specifically designed to handle large amounts amount of data.  But how does it accomplish this? In this lesson, we'll learn about the distributed nature of redshift, and how both the storage and query of data is distributed in redshift.

### Distributed Storage

Redshift is designed to store and query millions of rows of data.  To accomplish this, a single table may be partitioned across multiple nodes -- that is computer.  And even on the same node, a table may be partitioned across multiple slices.

<img src="./partitioned-data.png" width="100%">

This is whaat we see above.  Redshift has partitioned our `movies` table across four different slices, and two different compute nodes.  Within each slice, there is a data block for each column.  

> So five columns on each of four slices means a total of 20 data blocks, for 20 megabytes of storage.  

A compute node just means a separate computer where the data is stored.  A slice operates as a *virtual compute node* operating with it's own dedicated disk space and CPU resources.  A single slice on redshift cannot contain data from more than one table. 

### Parallel Queries

One of the benefits of partitioning our data across multiple slices, is that redshift can perform a single query in parallel on each slice.  For example, let's say we perform the following query: 

> `SELECT AVG(Year) FROM movies;`

Redshift will perform this query on each slice of the data and then aggregate the results.  Here's how this occurs.  

In addition to the compute nodes, Redshift has a leader node.  The leader node receives the query from the client, and then devises a plan to execute it across redshift's compute nodes.  

In this case, it will ask each slice to calculate the average movie year on it's own partition of the data.

<img src="./distributed-select.png" width="100%">

The leader then receives each slice's calculation and performs the final computation -- that is, it uses the results to calculate the average year across the dataset.   

### Reviewing the Architecture

So let's summarize what we learned about redshift's architecture.  Redshift consists of a single leader node and one or more compute nodes.

* **Leader node**

The leader node receives queries from external clients.  The leader node then determines the plan of attack for the compute nodes to execute the query.

* **Compute nodes**

The compute nodes are computers that consists of multiple slices.  Each slice is a *virtual compute node* with dedicated hard drive, memory and CPU resources.  Each slice queries it's own data, and operations can be performed across slices in parallel.

When the slices are done performing a query on their portion of the data, they send the results back to the leader node.  The leader node, may then perform some final computations -- like a final aggregation, or performing a sort or a limit of the data -- and then sends the results back to the user.

### Resources

[Redshift Deep Dive](https://www.youtube.com/watch?t=578&v=iuQgZDs-W7A&feature=youtu.be&ab_channel=AWSOnlineTechTalks)